In [ ]:
!pip install -q anthropic

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set


# Agent C: Build Your Own Agent!

Now it's your turn. You have the scaffold — fill in three things:
1. **System prompt** — Who is your agent? What's its personality?
2. **Tools** — What can it do? Define the JSON schemas.
3. **Dispatcher** — Implement each tool in Python.

The agentic loop is already written for you.

### Ideas
- **Travel assistant** — search flights, check visa requirements, convert currency
- **Recipe helper** — search recipes by ingredients, convert units, suggest substitutions
- **Study buddy** — create flashcards, quiz the user, explain concepts
- **Fitness coach** — log workouts, calculate calories, suggest exercises
- **Movie recommender** — search movies, get ratings, check streaming availability

Tools can return fake/hardcoded data — the point is designing the **interface** between the model and your code.

## Setup

In [ ]:
import anthropic

client = anthropic.Anthropic()

## TODO 1: Define your agent's persona

In [ ]:
SYSTEM_PROMPT = """You are a helpful assistant.

Replace this with your agent's persona and guidelines.
"""

## TODO 2: Define your tools

Each tool needs:
- `name` — a unique identifier (snake_case)
- `description` — tells Claude when to use it (be specific!)
- `input_schema` — a JSON Schema for the parameters

In [ ]:
tools = [
    # Replace this example with your own tools:
    {
        "name": "example_tool",
        "description": "Describe what this tool does so Claude knows when to use it",
        "input_schema": {
            "type": "object",
            "properties": {
                "param1": {
                    "type": "string",
                    "description": "Describe this parameter",
                }
            },
            "required": ["param1"],
        },
    },
    # Add more tools here...
]

## TODO 3: Implement your tools

In [ ]:
def call_tool(name, input):
    if name == "example_tool":
        return f"You called example_tool with: {input}"
    # Add more tool implementations here...
    return "Unknown tool"

## Run your agent

Once you've filled in the TODOs above, run this cell to test your agent!

In [ ]:
messages = [{"role": "user", "content": "Hello! What can you help me with?"}]

while True:
    response = client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=1024,
        system=SYSTEM_PROMPT,
        messages=messages,
        tools=tools,
    )

    messages.append({"role": "assistant", "content": response.content})

    tool_results = []
    for block in response.content:
        if block.type == "tool_use":
            result = call_tool(block.name, block.input)
            print(f"  [{block.name}] {block.input} → {result}")
            tool_results.append(
                {"type": "tool_result", "tool_use_id": block.id, "content": result}
            )
        elif block.type == "text" and response.stop_reason == "end_turn":
            print(f"\nAgent: {block.text}")

    if tool_results:
        messages.append({"role": "user", "content": tool_results})
    else:
        break

## Tips

- **Tool descriptions matter** — Claude uses them to decide when to call a tool. Be specific.
- **Start with 2-3 tools** — you can always add more later.
- **Fake data is fine** — the goal is designing the agent's interface, not building a production backend.
- **Test edge cases** — what happens when a tool returns an error? When the user asks something your tools can't handle?

For an interactive chat version:
```bash
uv run python exercises/09_agent_custom/agent_custom.py
```